In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
### All code blocks are either functions or classses so that its easier to debug
## Bad code is here

In [ ]:
%%time
## all imports
import dask
import pyarrow.parquet as pq
import dask.dataframe as dd
import os
import shutil
import json
from enum import Enum
from datetime import datetime
from ydata_profiling import ProfileReport
from pathlib import Path
import random


from dask.distributed import LocalCluster
##Metrics
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import mutual_info_score, adjusted_rand_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler


#For excel stuff
import openpyxl
from openpyxl.drawing.image import Image


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from shapely.geometry import Point

import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from pmdarima.arima.utils import ndiffs, nsdiffs
import numpy as np
from pmdarima import auto_arima
from sklearn.metrics import make_scorer, mean_absolute_percentage_error, mean_absolute_error,mean_squared_error
from pmdarima.metrics import smape
from pmdarima import model_selection
from neuralprophet import NeuralProphet, set_log_level
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose, STL, MSTL
import plotly.express as px
from prophet import Prophet



#from plotly_resampler import register_plotly_resampler
#register_plotly_resampler(mode='auto')


##h3
import h3

# Call the register function once and all Figures/FigureWidgets will be wrapped
# according to the register_plotly_resampler its `mode` argument
#register_plotly_resampler(mode='auto')

# Concurrency
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import time

#  Warnings
import warnings
warnings.filterwarnings('ignore')

# Set random state
random_state = 42
# Set figure size
plt.rcParams["figure.figsize"] = (20, 20)


parquet_file = {
    'merged':'/kaggle/input/road-accident-parquet/merged.parquet',
    2019: '/kaggle/input/road-accident-parquet/2019.parquet',
    2020: '/kaggle/input/road-accident-parquet/2020.parquet',
    2021: '/kaggle/input/road-accident-parquet/2021.parquet',
    2022: '/kaggle/input/road-accident-parquet/2022.parquet',
    2023: '/kaggle/input/road-accident-parquet/2023.parquet'
}


from dask.distributed import Client
client = Client()

pd.set_option('display.max_columns', None)



class ExtensionMethods:
    @staticmethod
    def generate_filename(filename=None,extension=None):
        current_datetime = datetime.now()
        f = current_datetime.strftime("%Y_%m_%d_%H%M")
        if (filename is None) or (extension is None):
            return str(f)
        else:
            stitched_f = str(filename)+"_"+str(f)+"."+str(extension)
            return str(stitched_f)

    @staticmethod
    def get_file_name_without_extension(filename):
        if filename == None:
            return "Provide a file"
        return Path(filename).stem


def create_parquet(data, filename='test'):
    if data is None:
        raise ValueError("Data can't be None for Parquet Creation")
    obj_cols = data.select_dtypes(include =['object','category'], exclude=['datetime','datetime64','datetimetz']).columns
    for col in obj_cols:
        data[col]=data[col].astype(str)
    filepath = os.path.join(os.getcwd(),ExtensionMethods.generate_filename(f"{filename}", "parquet"))
    data.to_parquet(filepath, engine='pyarrow',compression="zstd", compression_level=10, index=False)
    print(f"\n Finished Saving parquet to: {filepath}")




class CategoryBaseEnum(Enum):
    @classmethod
    def Name(cls):
        return f"{cls.__name__}"
    @classmethod
    def IsCategory(cls):
        return True if len(cls.__members__) > 0 else False
    @classmethod
    def get_description(cls):
        return cls.__doc__ or "No description available"
    @classmethod
    def to_dict(cls):
        return {member.name:member.value for member in cls}
    @classmethod
    def to_json(cls, indent=4):
        return json.dumps(cls.to_dict(), indent=indent)
    @classmethod
    def enum_keys(cls):
        return [member.name for member in cls]


class FileTypeEnum(Enum):
    @classmethod
    def Name(cls):
        return f"{cls.__name__}"
    @classmethod
    def get_description(cls):
        return cls.__doc__ or "No description available"
    @classmethod
    def to_dict(cls):
        return {member.name:member.value.to_dict() for member in cls}
    @classmethod
    def enum_keys(cls):
        return [member.name for member in cls]
    @classmethod
    def IsCategory(cls):
        return {member.name:member.value.IsCategory() for member in cls}
    @classmethod
    def MegaDictionary(cls):
        return {member.name:[member.value.to_dict(),member.value.IsCategory()] for member in cls}


class RoadAccidentEnum(FileTypeEnum):
    '''Collection of all the categories for file type 'vehicle.csv' '''

    class accident_id(CategoryBaseEnum):
        '''The Index/Number of the Crash follows the pattern yyyyxxxxx and is the index column'''

    class vehicle_id(CategoryBaseEnum):
        '''The vehicle id  in terms of xxx-xxx'''

    class vehicle_category(CategoryBaseEnum):  # catv Vehicle Category
        '''The Category of Vehicle involved in the crash'''
        UNKNOWN = 0
        LIGHT = 1
        MEDIUM = 2
        HEAVY = 3
        MISC = 4

    class obstacle_static(CategoryBaseEnum):  # obs Static ObstacleHit
        ''' The Static/Stationary Obstacle Hit'''
        UNKNOWN = 0
        NO_OBSTACLE = 1
        LIGHT = 2
        MEDIUM = 3
        HEAVY = 4

    class obstacle_mobile(CategoryBaseEnum):  # obsm #Mobile obstacle hit
        '''The Dynamic Obstacle Hit'''
        UNKNOWN = 0
        ANIMAL_OR_OTHER = 1
        VEHICLE = 2
        PEDESTRIAN = 3

    class impact_point(CategoryBaseEnum):  # choc Initial Point of Impact
        '''The Initial Point of Impact of the crash'''
        UNKNOWN = 0
        FRONT_IMPACT = 1
        SIDE_IMPACT = 2
        REAR_IMPACT = 3
        MULTIPLE_IMPACT = 4

    class action(CategoryBaseEnum):  # manv , Main action before crash
        '''The Main Action performed by the user before the crash'''
        UNKNOWN = 0
        STATIC = 1
        NORMAL_RISK = 2
        MEDIUM_RISK = 3
        HIGH_RISK = 4

    class motor(CategoryBaseEnum):  # motor
        '''The Type of Motor(In terms of fuel type) involved in the crash'''
        UNKNOWN = 0
        OTHER = 1
        TRADITIONAL = 2
        ELECTRIC_HYBRID = 3
        NON_MOTORIZED = 4

    class road(CategoryBaseEnum):
        '''Category of the road'''
        UNKNOWN = 0
        OTHER = 1
        URBAN_ROAD = 2
        HIGHWAY = 3

    class road_surface(CategoryBaseEnum):
        '''Surface condition'''
        UNKNOWN = 0
        NORMAL = 1
        MODERATE = 2
        SEVERE = 3

    class speed_limit(CategoryBaseEnum):
        '''Maximum speed permitted at the location and time of the accident.'''

    class user_category(CategoryBaseEnum):  #
        '''User category'''
        UNKNOWN = 0
        DRIVER = 1
        PASSENGER = 2
        PEDESTRIAN = 3

    class severity(CategoryBaseEnum):
        ''' Severity of the accident: The injured users are classified into three categories of victims plus the uninjured'''
        UNKNOWN = 0
        NO_INJURY = 1
        MINOR_INJURY = 2
        MAJOR_INJURY = 3
        KILLED = 4

    class sex(CategoryBaseEnum):
        '''User's gender'''
        UNKNOWN = 0
        MALE = 1
        FEMALE = 2

    class dob(CategoryBaseEnum):
        '''Year of birth of the user'''

    class safety_equipment(CategoryBaseEnum):
        '''The existence of a safety equipment'''
        UNKNOWN = 0
        NORMAL = 1
        GOOD = 2
        BEST = 3

    class datetime(CategoryBaseEnum):
        '''Date Time of the accident'''

    class lum(CategoryBaseEnum):
        '''Lighting conditions during the accident'''
        UNKNOWN = 0
        DAYLIGHT = 1
        NIGHT_LIT = 2
        LOW_LIGHT = 3
        NIGHT_DARK = 4

    class weather(CategoryBaseEnum):
        '''Weather conditions during the accident'''
        UNKNOWN = 0
        LIGHT = 1
        MEDIUM = 2
        SEVERE = 3

    class collision_type(CategoryBaseEnum):
        '''Type of collision'''
        UNKNOWN = 0
        NO_COLLISION = 1
        SIMPLE = 2
        COMPLEX = 3

    class lat(CategoryBaseEnum):
        '''Latitude of the accident location'''

    class long(CategoryBaseEnum):
        '''Longitude of the accident location'''

    class h3(CategoryBaseEnum):
        '''H3 Index of the accident'''

    class age(CategoryBaseEnum):
        '''Age of the person at time of accident'''

In [ ]:
## get 
def step1():
    valid_columns = RoadAccidentEnum.to_dict()
    valid_columns = valid_columns.keys()
    print(valid_columns)

    data = pd.read_parquet(parquet_file['merged'])

    data = data[[col for col in data.columns if col in valid_columns]]

    data['accident_hex_count'] = data.groupby('h3')['h3'].transform('count')
    data['date'] = pd.to_datetime(data['datetime']).dt.date
    return data


In [ ]:
def step2(data):
    duplicates = data[data.duplicated()]
    num_duplicates = duplicates.shape[0]
    print("Duplicate Check:")
    print(f"Number of duplicate rows: {num_duplicates}")
    if num_duplicates > 0:
        print("\nDuplicate Rows:")
        print(duplicates.head())
        print("\nDropping Duplicate Rows:")
        data = data.drop_duplicates()
    return data


In [ ]:
def step3(data):
    a = RoadAccidentEnum.MegaDictionary()
    replacement_dict={}
    for index, value in a.items():
        if a[index][1]==True:
            replacement_dict[index]=value[0]
    data = data.replace(replacement_dict).fillna(0)
    for index, value in replacement_dict.items():
        data[index] = pd.to_numeric(data[index])
        data[index] = data[index].replace(-1,0)
    return data

In [ ]:
def step4(data):
    columns_to_keep = [
    'impact_point', 'road_surface', 'speed_limit', 'user_category', 
    'severity', 'safety_equipment', 'lum', 'weather', 
    'collision_type', 'h3', 'accident_id', 'date','accident_hex_count']
    filtered_columns = [col for col in columns_to_keep if col in data.columns]
    aggregated_data = (
    data[filtered_columns]
    .groupby('date')
    .agg({
        'accident_id': 'count',
        'road_surface': 'max',
        'speed_limit': 'max',
        'lum': 'mean',
        'weather': 'max',
        'collision_type': 'max',
        'accident_hex_count':'mean'
    })
    .rename(columns={'accident_id': 'accident_count'})
    .reset_index())
    df = aggregated_data.copy()
    return df

In [ ]:
def step5(data):
    ##rename
    df = data
    df = df.rename(columns={'accident_count': 'y', 'date': 'ds'})
    df['ds'] = pd.to_datetime(df['ds'], format='%Y-%m-%d', errors='coerce')


    ## Seasonality
    df["summer"] = 0
    df.loc[df["ds"].dt.month.isin([6, 7, 8]), "summer"] = 1
    df["winter"] = 0
    df.loc[df["ds"].dt.month.isin([12, 1, 2]), "winter"] = 1
    df["fall"] = 0
    df.loc[df["ds"].dt.month.isin([9, 10, 11]), "fall"] = 1
    df["spring"] = 0
    df.loc[df["ds"].dt.month.isin([3, 4, 5]), "spring"] = 1

    df['weekend'] = 0
    df.loc[df['ds'].dt.dayofweek.isin([5, 6]), "weekend"] = 1
    return df

In [ ]:
def step6(df):
    end_date = pd.Timestamp('2023-06-30')
    mask = df['ds'] <= end_date
    train = df[mask]
    val = df[~mask]
    return train, val

In [ ]:
#https://medium.com/@roberto98russo/my-favorite-trend-seasonality-decomposition-algorithms-for-time-series-analysis-7fefabd717e5
def plot_components(result,what):
    df = pd.concat([result.observed, result.trend, result.seasonal, result.resid], axis=1)
    df = df.rename(columns={0: 'Original Data', 'season': 'seasonal', 'observed': 'Original Data'})
    
    df_long = df.stack().reset_index()
    df_long.columns = ['date', 'component', 'value']
    
    fig = px.line(df_long, 
                  x='date', 
                  y='value',
                  color='component',
                  facet_row='component',
                  facet_row_spacing=0.05,
                  height=1000,  
                  width=1000)   
    
    fig.update_layout(
        title=f'Time Series Decomposition-{what}',
        xaxis_title='Time',
        showlegend=True
    )
    
    fig.write_html(ExtensionMethods.generate_filename(f"Seasonal_Decompose_for_{what}",'html')) 

In [ ]:
##https://medium.com/towards-data-science/finding-seasonal-trends-in-time-series-data-with-python-ce10c37aa861
def time_series_analyser(df):
    ##TODO: Dont need it for the pipeline . Add later to autocreate graphs and pngs
    data = df[['ds', 'y']]
    data = data.set_index('ds')

    ##Perform stationary check
    result = adfuller(data['y'], autolag='AIC')
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print(f"  - Stationary: {'Yes' if result[1] < 0.05 else 'No'}")

    ## perform seasonal decompose and save png
    decompose = seasonal_decompose(data, model='additive')
    fig= decompose.plot()
    plt.tight_layout()
    plt.savefig(ExtensionMethods.generate_filename("Seasonal Decomposition_Additive",'png'))  
    plt.close(fig)
    print(f"The Variance for the additive model is {float(decompose.resid.var())}")
    
    decompose = seasonal_decompose(data, model='multiplicative')
    fig= decompose.plot()
    plt.tight_layout()
    plt.savefig(ExtensionMethods.generate_filename("Seasonal Decomposition_Multiplicative",'png'))  
    plt.close(fig)
    print(f"The Variance for the multiplicative model is {float(decompose.resid.var())}")

In [ ]:
def step_auto_arima(train, val):
    smape_scorer = make_scorer(smape, greater_is_better=False)
    mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

    model_arima = auto_arima(
        train[['y']],
        test='adf',
        m=12,
        seasonal_test='ocsb',
        start_P=0,
        seasonal=True,
        d=None,
        D=1,
        trace=True,  
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True,
        n_jobs=-1,
        maxiter=1
    )

    preds = model_arima.predict(n_periods=len(val))
    mape_score = mean_absolute_percentage_error(val['y'], preds)
    print(f"MAPE for time series score: {mape_score}")

    print("\n")
    print(model_arima.summary())

    fig = model_arima.plot_diagnostics()
    plt.tight_layout()
    plt.savefig(ExtensionMethods.generate_filename("Arima_Diagnostics_",'png'))  
    plt.close(fig)


    plt.plot(val['ds'], val['y'], label='Actual')
    plt.plot(val['ds'], preds, label='ARIMA Forecast')
    plt.title('Actual vs. Forecasted Accidents')
    plt.xlabel('Date')
    plt.ylabel('Score')
    plt.legend()
    plt.tight_layout()
    plt.savefig(ExtensionMethods.generate_filename("Arima_Forecast_",'png'))  
    plt.close()  
    
    return model_arima

In [ ]:
%%time
def step_neural_prophet(df):
    df = df[['ds','y','summer','winter','fall','spring','weekend','road_surface','weather','lum']]
    set_log_level("ERROR")

    m = NeuralProphet(
        n_changepoints=10,
        seasonality_mode='multiplicative',
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        n_lags=25
    )
    m.set_plotting_backend("matplotlib")  

    m = m.add_country_holidays(country_name="FR")
    m.add_lagged_regressor("road_surface", n_lags=1)
    m.add_future_regressor(name="weather")
    m.add_lagged_regressor("lum", n_lags=1)

    m.add_seasonality(name="summer", period=7, fourier_order=14, condition_name="summer")
    m.add_seasonality(name="winter", period=7, fourier_order=14, condition_name="winter")
    m.add_seasonality(name="spring", period=7, fourier_order=14, condition_name="spring")
    m.add_seasonality(name="fall", period=7, fourier_order=14, condition_name="fall")
    m.add_seasonality(name="weekend", period=1, fourier_order=6, condition_name="weekend")

    df_train, df_test = m.split_df(df, valid_p=0.1, local_split=True)
    print(f"Train shape: {df_train.shape}")
    print(f"Test shape: {df_test.shape}")

    metrics = m.fit(df_train, num_workers=3)
    forecast = m.predict(df)

    m.highlight_nth_step_ahead_of_each_forecast(1)
    
    fig = m.plot(forecast)
    plt.savefig(ExtensionMethods.generate_filename("Neural_Prophet_Plot_",'png'))  
    plt.close(fig)
    
    fig = m.plot_components(forecast)
    plt.savefig(ExtensionMethods.generate_filename("Neural_Prophet_Plot_Components",'png'))  
    plt.close(fig)
    
    fig = m.plot_parameters()
    plt.savefig(ExtensionMethods.generate_filename("Neural_Prophet_Plot_Parameters",'png'))  
    plt.close(fig)


    a = forecast.tail(len(df_test))
    mape_score = mean_absolute_percentage_error(df_test['y'], a['yhat1'])
    print(f"MAPE for time series score: {mape_score}")

    fig, ax = plt.subplots(figsize=(20,20))
    ax.plot(df_test['ds'], df_test['y'], label='Actual')
    ax.plot(df_test['ds'], a['yhat1'], label='Prophet Forecast')
    ax.legend()
    plt.savefig(ExtensionMethods.generate_filename("Neural_Prophet_Actual_vs_Forecast",'png'))
    plt.close(fig) 

    return m

In [ ]:
%%time
def step_prophet(train,val):
    train = train[['ds','y']] # prophet doenst like anything else, thats why neuralprophet is a bit better.
    val = val[['ds','y']]
    train['year'] = train['ds'].dt.year
    train['month'] = train['ds'].dt.month
    train['day'] = train['ds'].dt.day

    model = Prophet(changepoint_prior_scale=0.5,seasonality_mode='multiplicative') 
    model.add_country_holidays(country_name='FR')
    model.fit(train)

    future = model.make_future_dataframe(periods=len(val))
    future['year'] = future['ds'].dt.year
    future['month'] = future['ds'].dt.month
    future['day'] = future['ds'].dt.day

    forecast = model.predict(future)

    fig = model.plot(forecast)
    plt.title("Prophet Accident forecast")
    plt.xlabel("Date")
    plt.ylabel("Accidents")
    plt.legend()
    plt.tight_layout()
    plt.savefig(ExtensionMethods.generate_filename("Prophet_Accident_Forecast",'png'))
    plt.close(fig)

    fig = model.plot_components(forecast)
    plt.title("Prophet Model Future Components")
    plt.legend()
    plt.tight_layout()
    plt.savefig(ExtensionMethods.generate_filename("Prophet_Model_Future_Compoents",'png'))
    plt.close(fig)

    a = forecast.tail(len(val))
    mape_score = mean_absolute_percentage_error(val['y'], a['yhat'])
    print(f"MAPE for time series score: {mape_score}")

    fig, ax = plt.subplots(figsize=(20,20))
    ax.plot(val['ds'], val['y'], label='Actual')
    ax.plot(val['ds'], a['yhat'], label='Prophet Forecast')
    ax.legend()
    plt.savefig(ExtensionMethods.generate_filename("Prophet_Actual_vs_Forecast",'png'))
    plt.close(fig) 


In [ ]:
df= step1()
data = step5(step4(step3(step2(df)))) ## stupid but its fun to do bad code :)
time_series_analyser(data)

train,val = step6(data)
p_model = step_prophet(train,val)

arima_model = step_auto_arima(train,val)

neural_model = step_neural_prophet(data)